In [1]:
import pprint
import numpy as np
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

In [2]:
%matplotlib inline

## Global variables

In [3]:
DATA_DIR = '../data'
IS_BUSINESS_DAY = BDay().onOffset

### Reading and basic preprocessing of data

In [4]:
xau_cny = pd.read_csv(f'{DATA_DIR}/XAU_CNY_1Y.csv')
xau_cny = xau_cny.iloc[:, [0, 1]]

xau_cny['Price'] = xau_cny['Price'].str.replace(',', '').astype(float)

xau_cny = xau_cny.sort_values(by=['Date']).reset_index(drop=True)

print(f'There are {xau_cny.shape[0]} values for XAU/CNY\n')
print('Head values \n', xau_cny.tail(10), '\n')
print('Tail values\n', xau_cny.tail(10))

There are 1341 values for XAU/CNY

Head values 
               Date     Price
1331  Sep 28, 2015   7209.74
1332  Sep 28, 2016   8820.61
1333  Sep 28, 2017   8578.60
1334  Sep 28, 2018   8188.29
1335  Sep 29, 2015   7173.69
1336  Sep 29, 2016   8803.44
1337  Sep 29, 2017   8512.66
1338  Sep 30, 2015   7087.91
1339  Sep 30, 2016   8779.56
1340  Sep 30, 2019  10525.23 

Tail values
               Date     Price
1331  Sep 28, 2015   7209.74
1332  Sep 28, 2016   8820.61
1333  Sep 28, 2017   8578.60
1334  Sep 28, 2018   8188.29
1335  Sep 29, 2015   7173.69
1336  Sep 29, 2016   8803.44
1337  Sep 29, 2017   8512.66
1338  Sep 30, 2015   7087.91
1339  Sep 30, 2016   8779.56
1340  Sep 30, 2019  10525.23


In [5]:
usd_cny = pd.read_csv(f'{DATA_DIR}/USD_CNY_5Y.csv')
usd_cny = usd_cny.iloc[:, [0, 1]]

usd_cny = usd_cny.sort_values(by=['Date']).reset_index(drop=True)

print(f'There are {usd_cny.shape[0]} values for USD/CNY\n')
print('Head values \n', usd_cny.tail(10), '\n')
print('Tail values\n', xau_cny.tail(10))

There are 1301 values for USD/CNY

Head values 
               Date   Price
1291  Sep 28, 2015  6.3688
1292  Sep 28, 2016  6.6752
1293  Sep 28, 2017  6.6640
1294  Sep 28, 2018  6.8689
1295  Sep 29, 2015  6.3636
1296  Sep 29, 2016  6.6673
1297  Sep 29, 2017  6.6534
1298  Sep 30, 2015  6.3571
1299  Sep 30, 2016  6.6716
1300  Sep 30, 2019  7.1483 

Tail values
               Date     Price
1331  Sep 28, 2015   7209.74
1332  Sep 28, 2016   8820.61
1333  Sep 28, 2017   8578.60
1334  Sep 28, 2018   8188.29
1335  Sep 29, 2015   7173.69
1336  Sep 29, 2016   8803.44
1337  Sep 29, 2017   8512.66
1338  Sep 30, 2015   7087.91
1339  Sep 30, 2016   8779.56
1340  Sep 30, 2019  10525.23


In [ ]:
# Remove non-business days.
xau_cny_valid_dates = pd.to_datetime(xau_cny['Date']).map(IS_BUSINESS_DAY)
usd_cny_valid_dates = pd.to_datetime(usd_cny['Date']).map(IS_BUSINESS_DAY)

# Select business dates data.
xau_cny = xau_cny[xau_cny_valid_dates].reset_index(drop=True)
usd_cny = usd_cny[usd_cny_valid_dates].reset_index(drop=True)

print('Shape after preprocessing is :')
print('For XAU/CNY', xau_cny.shape)
print('For USD/CNY', usd_cny.shape)

In [ ]:
if usd_cny.shape[0] > xau_cny.shape[0]:
    A = usd_cny
    B = xau_cny
else:
    B = usd_cny
    A = xau_cny

In [ ]:
START_CONFIG = {
    'vault': 2000,
    'a': 1000,
    'b': 1000,
    'abp': 0,
    'bbp': 0
}

pp = pprint.PrettyPrinter(depth=6)

In [ ]:
START_CONFIG['abp'] = START_CONFIG.get('a') / A.loc[0, 'Price']

In [ ]:
START_CONFIG['bbp'] = START_CONFIG.get('b') / B.loc[0, 'Price']

In [ ]:
START_CONFIG

In [ ]:
a_prev = A.iloc[0, :]
b_prev = B.iloc[0, :]

for i in A.index.to_list():
    # Take the latest value or the last value.
    if i < A.shape[0]: a = A.loc[i, :]
    else: a = A.loc[A.index.to_list()[-1], :]
    
    if i < B.shape[0]: b = B.loc[i, :]
    else: b = B.loc[B.index.to_list()[-1], :]

    if a_prev.Price == a.Price and b_prev.Price == b.Price: continue
    
    # Get the new CNY values.
    new_config_a = START_CONFIG['abp'] * a.Price
    new_config_b = START_CONFIG['bbp'] * b.Price
    
    # Calculate the change in indiviaul CNY values.
    START_CONFIG['a'] = (new_config_a)
    START_CONFIG['b'] = (new_config_b)
    
    # START_CONFIG['a'] = START_CONFIG['abp'] * a.Price
    # START_CONFIG['b'] = START_CONFIG['bbp'] * b.Price
    
    # Calculate the diff between the 2 assets.
    diff = START_CONFIG['a'] - START_CONFIG['b']
    half_diff = diff / 2
    
    # Rebalance the assets.
    if diff == 0: continue
    else:
        START_CONFIG['a'] -= half_diff
        START_CONFIG['b'] += half_diff
        
    # Recalculate the buying power based on current price & current asset values.
    START_CONFIG['abp'] = START_CONFIG.get('a') / a.Price
    START_CONFIG['bbp'] = START_CONFIG.get('b') / b.Price
    
    # Recalculate the vault balance.
    START_CONFIG['vault'] = START_CONFIG.get('a') + START_CONFIG.get('b')
    
    print()
    pp.pprint({'Vault': START_CONFIG, 'Time': a.Date})
    print()
    
    if (START_CONFIG['vault'] - 2000) / 2000 <= -0.09: print('Breaking'); break
    
    a_prev, b_prev = a, b